In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import json

In [10]:
# !pip install tensorflow

In [11]:
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "martial-status",
        "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
        "hours-per-week", "native-country", "income"]
data = pd.read_csv('/Users/swithana/git/d2i/patra-toolkit/examples/notebooks/data/adult/adult.data', names=columns)

In [12]:
data.head()

,age,workclass,fnlwgt,education,education-num,martial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Pre-processing

In [13]:
label_encoder = LabelEncoder()
data['income'] = label_encoder.fit_transform(data['income'])

# Convert categorical variables to one-hot encoding
data = pd.get_dummies(data, drop_first=True, dtype=float)

data.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50,83311,13,0,0,13,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38,215646,9,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,53,234721,7,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,28,338409,13,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X = data.drop('income', axis=1).values
y = data['income'].values

print(data.columns.tolist())

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'income', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'martial-status_ Married-AF-spouse', 'martial-status_ Married-civ-spouse', 'martial-status_ Married-spouse-absent', 'martial-status_ Never-married', 'martial-status_ Separated', 'martial-status_ Widowed', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupa

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
X_train.shape

(26048, 100)

### Model Training

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - accuracy: 0.6608 - loss: 538.4090 - val_accuracy: 0.8058 - val_loss: 32.0143
Epoch 2/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - accuracy: 0.6800 - loss: 73.0914 - val_accuracy: 0.8015 - val_loss: 16.8599
Epoch 3/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 670us/step - accuracy: 0.6782 - loss: 27.4463 - val_accuracy: 0.7962 - val_loss: 9.2323
Epoch 4/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 694us/step - accuracy: 0.6853 - loss: 8.8401 - val_accuracy: 0.7954 - val_loss: 18.2872
Epoch 5/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 680us/step - accuracy: 0.6844 - loss: 7.4540 - val_accuracy: 0.7869 - val_loss: 3.1928
Epoch 6/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 681us/step - accuracy: 0.7136 - loss: 1.2877 - val_accuracy: 0.8035 - val_loss: 0.7474
Epoch 7/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 697us/step - accuracy: 0.7338 - loss: 0.9311 - val_accuracy: 0.8046 - val_loss: 0.6189
Epoch 8/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 697us/step - accuracy: 0.7483 - 

In [19]:
loss, accuracy = model.evaluate(X_test, y_test)

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - accuracy: 0.7824 - loss: 0.5178


In [20]:
y_pred = model.predict(X_test)

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step


## Model Card Generation

In [21]:
from patra_model_card.patra_model_card import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis

In [22]:
mc = ModelCard(
            name="UCI Adult Data Analysis model using Tensorflow",
            version="0.1",
            short_description="UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
            full_description="We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
            keywords="uci adult, tensorflow, explainability, fairness, patra",
            author="Sachith Withana",
            input_type="Tabular",
            category="classification"
        )

mc.input_data = 'https://archive.ics.uci.edu/dataset/2/adult'
mc.output_data = 'https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01'

In [23]:
ai_model = AIModel(
            name="Income prediction tensorflow model",
            version="0.1",
            description="Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
            owner="Sachith Withana",
            location="https://github.iu.edu/d2i/uci_adult/tensorflow_model",
            license="BSD-3 Clause",
            framework="tensorflow",
            foundational_model="None",
            model_type="dnn",
            test_accuracy=accuracy
        )
ai_model.populate_model_structure(model)

In [24]:
ai_model.add_metric("Test loss", loss)
ai_model.add_metric("Epochs", 100)
ai_model.add_metric("Batch Size", 32)
ai_model.add_metric("Optimizer", "Adam")
ai_model.add_metric("Learning Rate", 0.0001)
ai_model.add_metric("Input Shape", "(26048, 100)")

In [25]:
mc.ai_model = ai_model

In [26]:
mc.populate_requirements()

In [27]:
print(mc)

{
    "name": "UCI Adult Data Analysis model using Tensorflow",
    "version": "0.1",
    "short_description": "UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
    "full_description": "We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
    "keywords": "uci adult, tensorflow, explainability, fairness, patra",
    "author": "Sachith Withana",
    "input_type": "Tabular",
    "category": "classification",
    "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
    "output_data": "https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01",
    "ai_model": {
        "name": "Income prediction tensorflow model",
        "version": "0.1",
        "description": "Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
       

In [21]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).flatten()
mc.populate_bias(X_test, y_test, y_pred, "gender", X_test[:, 58], model)

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step


In [22]:
x_columns = data.columns.tolist()
x_columns.remove('income')

mc.populate_xai(X_test[:10], x_columns, model, 10)

In [23]:
print(mc.bias_analysis)

{'demographic_parity_diff': 0.015142125423592882, 'equal_odds_difference': 0.03403965947509895}


In [24]:
print(mc.xai_analysis)

{'capital_gain': 0.0925739695959621, 'fnlwgt': 0.0462870226138168, 'hours_per_week': 8.112854433004222e-10, 'age': 4.6359168140133437e-10, 'occupation__Prof_specialty': 2.649095322733904e-10, 'education_num': 2.31795839929679e-10, 'education__Masters': 2.152389952997997e-10, 'education__HS_grad': 1.3245476598249755e-10, 'occupation__Exec_managerial': 9.93410742555767e-11, 'workclass__Private': 6.622738299124877e-11}


In [25]:
print(mc)

{
    "name": "UCI Adult Data Analysis model using Tensorflow",
    "version": "0.1",
    "short_description": "UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
    "full_description": "We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
    "keywords": "uci adult, tensorflow, explainability, fairness, patra",
    "author": "Sachith Withana",
    "input_type": "Tabular",
    "category": "classification",
    "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
    "output_data": "https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01",
    "ai_model": {
        "name": "Income prediction tensorflow model",
        "version": "0.1",
        "description": "Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
       

In [26]:
mc.save("/Users/swithana/git/d2i/patra-toolkit/examples/model_cards/tesorflow_adult_nn_MC.json")

In [27]:
mc.validate()

True

In [28]:
mc.submit("http://127.0.0.1:5000/upload_mc")

{'message': 'Successfully uploaded the model card',
 'model_card_id': 'd9f7aa3e-a9ac-4084-a5cc-92cfaf4eacde'}